## Import packages

In [1]:
import pandas as pds
import numpy as np
import textdistance
import timeit
import math
import statsmodels.api as sm
import scipy
from patsy import dmatrix
import time
import math

## Load dataset

In [2]:
name_DF = 'DF_N=4401_2023-01-16.csv'
DF = pds.read_csv(os.path.join('..', 'datasets', name_DF), delimiter = ',')
DF = DF[~DF.duplicated()] # delete duplicates
DF = DF.dropna() # delete NaN values
DF['was_assigned_female'] = DF['was_assigned_female'].astype('int32') # turn was_born_female into int type (once Nan values have been removed)

identifiers = {'family_name':'jaro-winkler','was_assigned_female':'strict','country':'strict','birth_year':'large'}
covariates = ['X1','X2','X3','X4','X5']

## Generate association

$Y = -10 + a T X_2 + b \exp^{X_4} + c X_3 X_1 + d X_5$

Then the average treatment effect equals $a \times \mathbb E [X_2]$

In [3]:
########## GENERATES ASSOCIATION ##########

# generate covariates
DF['X1'] = 2020 - DF['birth_year'] # age
DF['X2'] = np.random.normal(loc = 2.5, scale = 1, size = DF.shape[0])
DF['X3'] = np.random.normal(loc = 0, scale = 1, size = DF.shape[0])
DF['X4'] = np.random.normal(loc = 1, scale = 1, size = DF.shape[0])
DF['X5'] = np.random.normal(loc = 1, scale = 1, size = DF.shape[0])

# generate treatment
DF['treatment'] = np.random.binomial(n = 1, p = 1 / ( 1 + np.exp(0.1*DF.X1 -0.2*DF.X2 +0.3*DF.X3 -0.4*DF.X4 +0.5*DF.X5) )) # probability depending on covariates

# generate outcome
residual_errors = np.random.normal(size = DF.shape[0])
a = 5.5
b = 0.01
c = 0.08
d = 0.7

ate = a * 2.5
DF['Y'] = - 10 + a*DF['treatment']*DF['X2'] + b*np.exp(DF['X4']) + c*DF['X3']*DF['X1'] + d*DF['X5'] 

## Generate 2 datasets with overlap for record linkage

In [4]:
common_records = DF.sample(n = 800)

B = pds.concat([DF.sample(n = 1400), common_records]).drop(['Y'], axis = 1)
B = B.reset_index(drop=True)

A = pds.concat([DF.sample(n = 2000), common_records])[list(identifiers.keys())+['Y']]
A = A.reset_index(drop=True)

## Build functions

- comparison_vector
- linking_score
- levenshtein_similarity
- jaro_winkler_similarity
- strict_equality
- large_equality
- logit
- minmaxscaler
- propensity_score

In [5]:
# def comparison_vector(A_record, B, identifiers):
    
#     """ Compare one record in A with all records in B. 
#         Return the binary comparison of the identifiers for one record in A with all records in B.

#         A_record:     series of one row, 
#         B:            dataframe, 
#         identifiers:  dict: k = column name, 
#                             v = method in {'large','strict','levenshtein','jaro-winkler'}
#     """

#     methods = {'jaro-winkler':jaro_winkler_similarity, 'levenshtein':levenshtein_similarity, 'strict':strict_equality, 'large':large_equality}
#     comparisons = {}
#     for linking_var in identifiers:
#         method = methods[identifiers[linking_var]]
#         comparisons[linking_var] = np.array(B.apply(lambda row: method(A_record[linking_var], row[linking_var]), axis=1)).reshape(-1,1)
#     return np.concatenate(tuple(comparisons.values()), axis = 1) 

# def linking_score(A, B, identifiers, match, unmatch):
        
#     """ Compare records in A with records in B, computing all linking scores for records in A with records in B. 
#         Return the indices of records in A with the best match index for record in B.

#         A:            dataframe, 
#         B:            dataframe, 
#         identifiers:  dict: k = column name, 
#                             v = method in {'large','strict','levenshtein','jaro-winkler'}
#         match:        array of probabilities of having same linking variables when being a match,
#         unmatch:      array of probabilities of having same linking variables (at all, among the nA x nB pairs of record).
#     """

#     def compute_max_linking_score(A_record, B, identifiers, match, unmatch):
#         similarities = comparison_vector(A_record, B, identifiers)
#         if similarities.all(axis=1).sum() == 1: # there is one unique match: possible true match
#             return B.iloc[similarities.all(axis=1),:].index[0], np.NaN
#         else:
#             linking_score = (np.multiply(similarities, np.log2(match/unmatch)) + np.multiply(1-similarities, np.log2((1-match)/(1-unmatch)))).sum(axis=1)
#             return linking_score.argmax(), linking_score.max() # assign the first element of the argmax set A REVOIR

#     links = A.apply(lambda row: compute_max_linking_score(row[list(identifiers.keys())], B, identifiers, match, unmatch), axis=1)
#     idx_in_A = np.arange(A.shape[0])
#     idx_in_B = np.array([element[0] for element in links])
#     matching_scores = np.array([element[1] for element in links])
#     max_score = np.nanmax(matching_scores)
#     matching_scores = np.nan_to_num(matching_scores, nan=max_score+1) # possible true matches are given a higher score
#     return {'A':idx_in_A, 'B':idx_in_B, 'scores':matching_scores}
    
def levenshtein_similarity(a,b):

    """ Check that levenshtein similarity (in [0,1]) is above 0.95.
        
        a: string,
        b: string """

    if 1 - textdistance.levenshtein(a, b)/max(len(a),len(b)) >= 0.95:
        return 1
    else:
        return 0

def jaro_winkler_similarity(a,b):

    """ Check that jaro-winkler similarity (in [0,1]) is above 0.95.
        
        a: string,
        b: string """

    if textdistance.jaro_winkler(a,b) >= 0.99:
        return 1
    else:
        return 0

def strict_equality(a,b):

    """ Check that a and b values are equal.
        
        a: any value,
        b: any value """

    return a==b

def large_equality(a,b):

    """ Check that years a and b expressed with four numbers are within the same decade.
        
        a: year,
        b: year """

    return str(a)[:-1]==str(b)[:-1]

def logit(p):
    return np.log(p/(1-p))

def minmaxscaler(X):
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    
def propensity_score(DF, covariates, scaler, convert_to_logit):
    
    """ Compute propensity score estimates: the probability (logistic regression) that an observation is treated or not conditioned on some covariates.
        These estimates are built conditionaly on covariates passed using a logit after transformation by scaler (when one is specified).
        Estimated probabilities can be converted into logit (convert_to_logit parameter).

        DF:                dataframe,
        covariates:        list of strings for covariates variable in DF,
        scaler:            sklearn.preprocessing function scaler for exemple,
        convert_to_logit:  boolean for converting probabilities to logit when building the propensity score estimates based on a logistic regression
    """
    exog = covariates.copy()
    if scaler != None:
        DF[exog] = scaler(DF[exog])
    if 'intercept' not in DF.columns:
        DF['intercept'] = 1
    exog.append('intercept')
    model = sm.Logit(DF.treatment, DF[exog]).fit()
    predictions = model.predict(DF[exog])
    if convert_to_logit:
        return logit(predictions)
    else: 
        return predictions

## Replicate Guha's work

### Generate initial parameters

- z
- linkage match
- linkage unmatch
- outcome match
- outcome unmatch

#### Build initial linkage parameter

- z

In [6]:
# z_0

# for each index (in A) 
# z = index of the record linked in B or if not, n_A + index

# z = np.arange(n_A)
# matchings = linking_score(A, B, identifiers, match, unmatch)
# best_score = np.max(matchings['scores']) 
# best_matches = matchings['scores'] >= best_score
# z[best_matches] = matchings['B'][best_matches]
# z[~best_matches] = z[~best_matches] + n_A
# print(f"# common records:  {common_records.shape[0]}\n# best matches:    {len(z[z<=n_A])}")

#### Build initial set of linked records

In [7]:
# idx_in_A = np.nonzero(z[z <= n_A])[0]
# idx_in_B = z[z <= n_A]
# from_A = A.iloc[idx_in_A,:].reset_index(drop=True)
# from_B = B.iloc[idx_in_B,:].reset_index(drop=True)
# linked_records = pds.concat([from_A, from_B.Y], axis=1)
# linked_records['propensity_score'] = propensity_score(linked_records, covariates, None, False)
# print(f"\n# common records:  {common_records.shape[0]}\n# linked records:  {linked_records.shape[0]}\n")
# linked_records.head()

#### Build parameters for match and unmatch distr. of linkage variables and a table of the comparison btw $A$ and $B$ records

- linkage match
- linkage unmatch
- table of similarities among records???

In [8]:
# z = np.arange(A.shape[0])
# z[data["source_index_A"]]

Create all possible pairs of records in $AB$

In [9]:
AB = B[identifiers.keys()].merge(A[identifiers.keys()], how='cross')

AB["source_index_B"] = np.repeat(B.index, A.shape[0])
AB["source_index_A"] = np.tile(A.index, B.shape[0])

methods = {'jaro-winkler':jaro_winkler_similarity, 'levenshtein':levenshtein_similarity, 'strict':strict_equality, 'large':large_equality}

for linking_var in identifiers.keys():
    method = methods[identifiers[linking_var]]
    df = AB.filter(regex=linking_var)
    AB[linking_var+"_comparison"] = np.array([method(a, b) for a,b in zip(df.iloc[:,0], df.iloc[:,1])]).astype(int).reshape(-1,1)

AB

,family_name_x,was_assigned_female_x,country_x,birth_year_x,family_name_y,was_assigned_female_y,country_y,birth_year_y,source_index_B,source_index_A,family_name_comparison,was_assigned_female_comparison,country_comparison,birth_year_comparison
0,Hofer,0,CH,2008,Hols,0,IT,2013,0,0,0,1,0,0
1,Hofer,0,CH,2008,Kral,1,AT,1952,0,1,0,0,0,0
2,Hofer,0,CH,2008,Nazari,0,DE,1995,0,2,0,1,0,0
3,Hofer,0,CH,2008,Uter,0,DE,2003,0,3,0,1,0,1
4,Hofer,0,CH,2008,Иванова,1,RU,1998,0,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6159995,Marchi,0,IT,1948,Воронцова,1,RU,1976,2199,2795,0,0,0,0
6159996,Marchi,0,IT,1948,Arjona Gomez,1,ES,1981,2199,2796,0,0,0,0
6159997,Marchi,0,IT,1948,Ådland,1,NO,2014,2199,2797,0,0,0,0
6159998,Marchi,0,IT,1948,Губатюк,1,RU,1955,2199,2798,0,0,0,0


Select only "exact" matches (where all comparisons $= 1$ and enforce 1-2-1 matching removing all duplicata)

In [10]:
data = AB[AB.filter(regex="comparison").all(axis=1)]
comparisons_exact_matched_records = data[ (~data.filter(regex="_x").duplicated(keep=False)) & (~data.filter(regex="_y").duplicated(keep=False)) ]
comparisons_exact_matched_records

,family_name_x,was_assigned_female_x,country_x,birth_year_x,family_name_y,was_assigned_female_y,country_y,birth_year_y,source_index_B,source_index_A,family_name_comparison,was_assigned_female_comparison,country_comparison,birth_year_comparison
670,Hofer,0,CH,2008,Hofer,0,CH,2008,0,670,1,1,1,1
9473,Madjidov,0,RU,1976,Madjidov,0,RU,1976,3,1073,1,1,1,1
14872,Thomassen,0,NO,2014,Thomassen,0,NO,2014,5,872,1,1,1,1
31559,Кузнецов,0,RU,1955,Кузнецов,0,RU,1955,11,759,1,1,1,1
37186,Sweet,0,GB,1982,Sweet,0,GB,1982,13,786,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6126387,Vanderhoeden,1,BE,2002,Vanderhoeden,1,BE,2002,2187,2787,1,1,1,1
6137591,Kopotilov,0,RU,1961,Kopotilov,0,RU,1961,2191,2791,1,1,1,1
6140392,Мур,1,RU,1955,Мур,1,RU,1955,2192,2792,1,1,1,1
6151596,Arjona Gomez,1,ES,1981,Arjona Gomez,1,ES,1981,2196,2796,1,1,1,1


Compute linking score over all pairs of records

In [11]:
comparisons = AB.filter(regex="comparison")

unmatch = comparisons.sum(axis=0) / len(comparisons) # probability of having same linking var (at all)

match = np.repeat(0.95, len(identifiers.keys())) # probability of having same linking var when being matches

AB["linking_score"] = (np.multiply(comparisons, np.log2(match/unmatch)) + np.multiply(1-comparisons, np.log2((1-match)/(1-unmatch)))).sum(axis=1)

AB

,family_name_x,was_assigned_female_x,country_x,birth_year_x,family_name_y,was_assigned_female_y,country_y,birth_year_y,source_index_B,source_index_A,family_name_comparison,was_assigned_female_comparison,country_comparison,birth_year_comparison,linking_score
0,Hofer,0,CH,2008,Hols,0,IT,2013,0,0,0,1,0,0,-11.639935
1,Hofer,0,CH,2008,Kral,1,AT,1952,0,1,0,0,0,0,-15.886268
2,Hofer,0,CH,2008,Nazari,0,DE,1995,0,2,0,1,0,0,-11.639935
3,Hofer,0,CH,2008,Uter,0,DE,2003,0,3,0,1,0,1,-4.867929
4,Hofer,0,CH,2008,Иванова,1,RU,1998,0,4,0,0,0,0,-15.886268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6159995,Marchi,0,IT,1948,Воронцова,1,RU,1976,2199,2795,0,0,0,0,-15.886268
6159996,Marchi,0,IT,1948,Arjona Gomez,1,ES,1981,2199,2796,0,0,0,0,-15.886268
6159997,Marchi,0,IT,1948,Ådland,1,NO,2014,2199,2797,0,0,0,0,-15.886268
6159998,Marchi,0,IT,1948,Губатюк,1,RU,1955,2199,2798,0,0,0,0,-15.886268


For each record in $A$ we would like to find the best match in $B$ (we remove records from $A$ and from $B$ that matches more than once)

Produce the linkage variable $$z_i = \left\{
    \begin{array}{ll}
        j & \text{if } \{i,j\} \text{ are matched} \\
        i + n_B & \text{otherwise.}
    \end{array}
\right.$$

In [12]:
score = AB.linking_score.max()
data = AB[AB.linking_score==score]

# we consider 1-2-1 matches in z
data_link = data[ (~data.source_index_A.duplicated(keep=False)) & (~data.source_index_B.duplicated(keep=False)) ]
z = -np.ones(B.shape[0])
z[data_link.source_index_B] = data_link.source_index_A

data_nolink = AB[AB.linking_score!=score]
# build comparisons, pattern and count for linked records:
comparisons_match = data_link.filter(regex="comparison")
pattern_match, count_match = np.unique(comparisons_match, return_counts=True, axis=0)

# build comparisons, pattern and count for non linked records (from A):
comparisons_unmatch = data_nolink.filter(regex="comparison")
pattern_unmatch, count_unmatch = np.unique(comparisons_unmatch, return_counts=True, axis=0)

from_A = A.iloc[data_link.source_index_A,:].reset_index(drop=True)
from_B = B.iloc[data_link.source_index_B,:].reset_index(drop=True)
linked_records = pds.concat([from_B, from_A.Y], axis=1)
linked_records['propensity_score'] = propensity_score(linked_records, covariates, None, False)
linked_records

Optimization terminated successfully.
         Current function value: 0.240693
         Iterations 8


,name,family_name,country,birth_year,was_assigned_female,X1,X2,X3,X4,X5,treatment,Y,intercept,propensity_score
0,Stefan,Hofer,CH,2008,0,12,2.690921,-0.114563,1.308119,1.468779,1,5.755224,1,0.342631
1,Mykhammad,Madjidov,RU,1976,0,44,3.873064,-0.522982,0.927945,0.507575,0,-11.460300,1,0.030170
2,Gunnar Wessel,Thomassen,NO,2014,0,6,2.155654,-0.359078,1.023478,3.098727,0,-7.975420,1,0.299145
3,Евгений,Кузнецов,RU,1955,0,65,1.188225,-0.571403,1.572769,2.455389,0,-11.204325,1,0.001194
4,Aj,Sweet,GB,1982,0,38,0.576695,1.887227,-0.312163,1.189375,0,-3.422948,1,0.013648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Raymonde,Vanderhoeden,BE,2002,1,18,1.858974,1.078054,0.233022,1.575420,0,-7.332183,1,0.129250
806,Anton,Kopotilov,RU,1961,0,59,3.169091,-1.398690,0.716948,-0.226947,0,-16.740199,1,0.008720
807,Скромница,Мур,RU,1955,1,65,2.314127,1.101916,2.095628,0.905180,0,-3.555105,1,0.002430
808,Noelia,Arjona Gomez,ES,1981,1,39,2.766746,-0.586073,1.703235,0.583568,0,-11.365133,1,0.051485


#### First element in the likelihood: conditional outcome distribution for matches

In [13]:
Y = np.array(linked_records.Y)

X = np.array([linked_records.propensity_score*linked_records.treatment, linked_records.propensity_score])
X = sm.add_constant(X)
X = X.T

model = sm.GLM(Y,X)
results = model.fit()
print(results.summary())

residuals = Y - X @ results.params

estimated_variance = residuals.T @ residuals / (len(residuals) - (X.shape[1]+1))

np.log(scipy.stats.norm.pdf(residuals, 0, np.sqrt(estimated_variance))).sum()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  810
Model:                            GLM   Df Residuals:                      808
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                          69.452
Method:                          IRLS   Log-Likelihood:                -2865.8
Date:                Tue, 31 Jan 2023   Deviance:                       56117.
Time:                        20:34:42   Pearson chi2:                 5.61e+04
No. Iterations:                     3   Pseudo R-squ. (CS):            -0.5790
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            33.1088      2.322     14.259      0.0

-2865.8005608669273

#### Second element in the likelihood: conditional outcome distribution for non matches

In [14]:
# log likelihood 2

a_sigma2 = 1
b_sigma2 = 1
mu2 = scipy.stats.norm.rvs(0, 1)
sigma2_square = scipy.stats.invgauss.rvs(a_sigma2, b_sigma2)

Y = scipy.stats.norm.rvs(mu2, np.sqrt(sigma2_square), size = A.shape[0]-linked_records.shape[0])

np.log(scipy.stats.norm.pdf(Y, mu2, np.sqrt(sigma2_square))).sum()

-3507.93472520142

#### Third element in the likelihood: linkage distribution for matches

In [15]:
# likelihood 3

((pattern_match @ np.log(match) + (1-pattern_match) @ np.log(1-match)) * count_match).sum()

-166.1902738156639

#### Fourth element in the likelihood: linkage distribution for unmatches

In [16]:
# likelihood 4

((pattern_unmatch @ np.log(unmatch) + (1-pattern_unmatch) @ np.log(1-unmatch)) * count_unmatch).sum()

-8987576.199955655

In [17]:
# OUTCOME MODEL SPLINE REGRESSION ????????????????????????????????????????????????????

# N = linked_records.shape[0]

# s = 10
# m = 20

# beta_0 = scipy.stats.norm.rvs(0, 1, size=N)

# betas_s = scipy.stats.norm.rvs(0, 1, size=(N,s))

# gammas_s = scipy.stats.norm.rvs(0, 1, size=(N,s))

# a_sigma = 1
# b_sigma = 1
# sigma_square = scipy.stats.invgauss.rvs(a_sigma, b_sigma)

# knots = scipy.stats.uniform.rvs(0, 1, size=m) # remain unchanged
# knots.sort()

# r1 = 1
# delta1 = 1
# lambda1_square = scipy.stats.gamma.rvs(r1, delta1)
# tau1_m_square = scipy.stats.expon.rvs(np.sqrt(lambda1_square), size=(N,m))
# betas_s_m = scipy.stats.norm.rvs(0, np.sqrt(sigma_square*tau1_m_square), size=(N,m))

# r2 = 1
# delta2 = 1
# lambda2_square = scipy.stats.gamma.rvs(r2, delta2)
# tau2_m_square = scipy.stats.expon.rvs(np.sqrt(lambda2_square), size=(N,m))
# gammas_s_m = scipy.stats.norm.rvs(0, np.sqrt(sigma_square*tau2_m_square), size=(N,m))

# # for each record in linked_records: we build its outcome based on a regression splines
# outcome_distr_match = beta_0 + ((betas_s+(np.tile(linked_records.treatment, (s,1)).T)*gammas_s) * (np.tile(linked_records.propensity_score, (s,1)).T)**np.arange(1,s+1)).sum(axis=1) + ((betas_s_m+(np.tile(linked_records.treatment, (m,1)).T)*gammas_s_m) * np.maximum(0,(np.tile(linked_records.propensity_score, (m,1)).T - knots))**np.arange(1,m+1)).sum(axis=1) + scipy.stats.norm.rvs(0, 1, size=N)

## METROPOLIS HASTINGS

In [18]:
start = time.time()

In [19]:
# INITIALISATION

z_k = [z.copy()]

linked_records_k = [linked_records.copy()]

theta_m_k = [match.copy()]

theta_u_k = [unmatch.copy()]

alpha_pi_k = [1]
beta_pi_k = [1]

a_sigma_k = [1]
b_sigma_k = [1]

a_sigma2_k = [1]
b_sigma2_k = [1]

beta0_k = [scipy.stats.norm.rvs(0,1)]
beta1_k = [scipy.stats.norm.rvs(0,1)]
alpha_k = [scipy.stats.norm.rvs(0,1)]
sigma_square_k = [scipy.stats.invgauss.rvs(a_sigma_k[-1],b_sigma_k[-1])]

mu2_k = [scipy.stats.norm.rvs(0,1)]
sigma2_square_k = [scipy.stats.invgauss.rvs(a_sigma2_k[-1],b_sigma2_k[-1])]

posteriors = []

In [20]:
# COMPUTE POSTERIOR

def compute_posterior(linked_records, z, theta_m, theta_u, alpha_pi, beta_pi, sigma_square, a_sigma, b_sigma, beta0, beta1, alpha, mu2, sigma2_square, a_sigma2, b_sigma2):
    result = 0
    # likelihood 1
    # need linked_records computed based on z
    Y = np.array(linked_records.Y)
    X = np.array([linked_records.propensity_score*linked_records.treatment, linked_records.propensity_score])
    X = sm.add_constant(X)
    X = X.T
    model = sm.GLM(Y,X)
    results = model.fit()
    residuals = Y - X @ results.params
    estimated_variance = residuals.T @ residuals / (len(residuals) - (X.shape[1]+1))
    result += np.log(scipy.stats.norm.pdf(residuals, 0, np.sqrt(estimated_variance))).sum()
    # likelihood 2
    # need z
    Y = scipy.stats.norm.rvs(mu2, np.sqrt(sigma2_square), size=len(z[z<0]))
    result += np.log(scipy.stats.norm.pdf(Y, mu2, np.sqrt(sigma2_square))).sum()
    # likelihood 3 and 4
    # need AB, z, theta_m, theta_u ATTENTION AB SHOULD BE KNOWN (GLOBAL)
    idx_A = z[z>=0]
    idx_B = np.nonzero(z>=0)[0]
    links = pds.MultiIndex.from_tuples(zip(idx_A,idx_B))
    pairs = pds.MultiIndex.from_frame(AB[["source_index_A", "source_index_B"]])
    # 3
    data = AB[pairs.isin(links)] # 1-2-1 matches enforced by construction of z
    pattern_match, count_match = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
    result += ((pattern_match @ np.log(theta_m) + (1-pattern_match) @ np.log(1-theta_m)) * count_match).sum()
    # 4
    data = AB[(~AB.source_index_B.duplicated())&(~pairs.isin(links))] # enforce 1-2-1 by removing duplicata
    pattern_unmatch, count_unmatch = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
    result += ((pattern_unmatch @ np.log(theta_u) + (1-pattern_unmatch) @ np.log(1-theta_u)) * count_unmatch).sum()
    # prior 1
    # need z, alpha_pi, beta_pi
    n_AB = (z>=0).sum()
    result += math.log(math.factorial(A.shape[0]-n_AB)) - math.log(math.factorial(A.shape[0])) + scipy.special.betaln(n_AB + alpha_pi, B.shape[0] - n_AB + beta_pi) - scipy.special.betaln(alpha_pi, beta_pi)
    # prior 2
    # need theta_m QUESTION a-1 / b-1?
    result += (1 * np.log(theta_m) + 1 * np.log(1-theta_m)).sum()
    # prior 3
    # need theta_u QUESTION a-1 / b-1?
    result += (1 * np.log(theta_u) + 1 * np.log(1-theta_u)).sum()
    # prior 4
    # need sigma_square, a_sigma, b_sigma QUESTION unclear if it is the pdf?
    result += np.log(scipy.stats.invgauss.pdf(sigma_square, a_sigma, b_sigma))
    # prior 5
    # need beta0, beta1, alpha QUESTION unclear if it is the pdf?
    result += np.log(scipy.stats.multivariate_normal.pdf([beta0, beta1, alpha], [0,0,0], np.eye(3)))
    # prior 6
    # need mu2 QUESTION unclear if it is the pdf?
    result += np.log(scipy.stats.norm.pdf(mu2, 0, 1))
    # prior 7
    # need sigma2_square, a_sigma2, b_sigma2 QUESTION unclear if it is the pdf?
    result += np.log(scipy.stats.invgauss.pdf(sigma2_square, a_sigma2, b_sigma2))
    return result

posterior = compute_posterior(linked_records_k[-1], z_k[-1], theta_m_k[-1], theta_u_k[-1], alpha_pi_k[-1], beta_pi_k[-1], sigma_square_k[-1], a_sigma_k[-1], b_sigma_k[-1], beta0_k[-1], beta1_k[-1], alpha_k[-1], mu2_k[-1], sigma2_square_k[-1], a_sigma2_k[-1], b_sigma2_k[-1])
posteriors.append(posterior)

In [21]:
# COMPUTE PROPOSAL

def compute_proposal(z, sigma_square, a_sigma, b_sigma, beta0, beta1, alpha, mu2, sigma2_square, a_sigma2, b_sigma2):
    result = [] # z, linked_records, theta_m, theta_u, beta0, beta1, alpha, sigma_square, mu2, sigma2_square
    # z
    linked_record_prop = (z<B.shape[0]).sum() / len(z) # proportion of linked records in previous z
    new = np.random.choice(A.shape[0], size=B.shape[0], replace=False)
    new = new * np.random.choice([1,-1], size=B.shape[0], p=[linked_record_prop, 1-linked_record_prop]) # randomly set negative some values so that the proportion of positive values corresponds to linked_record_prop
    result.append(new)
    # linked records
    idx_A = z[z>=0]
    idx_B = np.nonzero(z>=0)[0]
    from_A = A.iloc[idx_A,:].reset_index(drop=True)
    from_B = B.iloc[idx_B,:].reset_index(drop=True)
    linked_records = pds.concat([from_B, from_A.Y], axis=1)
    linked_records['propensity_score'] = propensity_score(linked_records, covariates, None, False)
    result.append(linked_records)
    # theta_m QUESTION we do not care of dependence on previous param
    links = pds.MultiIndex.from_tuples(zip(idx_A,idx_B))
    pairs = pds.MultiIndex.from_frame(AB[["source_index_A", "source_index_B"]])
    comparisons = AB[pairs.isin(links)].filter(regex="comparison")
    match = comparisons.sum(axis=0) / len(comparisons)
    result.append(match)
    # theta_u QUESTION we do not care of dependence on previous param
    comparisons = AB.filter(regex="comparison")
    unmatch = comparisons.sum(axis=0) / len(comparisons)
    result.append(unmatch)
    # beta0, beta1, alpha, sigma_square
    result.append(beta0+scipy.stats.norm.rvs(0,0.5))
    result.append(beta1+scipy.stats.norm.rvs(0,0.5))
    result.append(alpha+scipy.stats.norm.rvs(0,0.5))
    result.append(sigma_square+scipy.stats.invgauss.rvs(a_sigma,b_sigma))
    # mu2, sigma2_square
    result.append(mu2+scipy.stats.norm.rvs(0,1))
    result.append(sigma2_square+scipy.stats.invgauss.rvs(a_sigma2,b_sigma2))
    return result

proposal = compute_proposal(z_k[-1], sigma_square_k[-1], a_sigma_k[-1], b_sigma_k[-1], beta0_k[-1], beta1_k[-1], alpha_k[-1], mu2_k[-1], sigma2_square_k[-1], a_sigma2_k[-1], b_sigma2_k[-1])
new_z, new_linked_records, new_theta_m, new_theta_u, new_beta0, new_beta1, new_alpha, new_sigma_square, new_mu2, new_sigma2_square = proposal

new_posterior = compute_posterior(new_linked_records, new_z, new_theta_m, new_theta_u, alpha_pi_k[-1], beta_pi_k[-1], new_sigma_square, a_sigma_k[-1], b_sigma_k[-1], new_beta0, new_beta1, new_alpha, new_mu2, new_sigma2_square, a_sigma2_k[-1], b_sigma2_k[-1])
posteriors.append(new_posterior)

Optimization terminated successfully.
         Current function value: 0.240693
         Iterations 8


/Users/kayane/anaconda3/envs/tester/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kayane/anaconda3/envs/tester/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
U = scipy.stats.uniform.rvs(0,1)
if U < posteriors[-1] / posteriors[-2]:
    z_k.append(new_z)
    linked_records_k.append(new_linked_records)
    theta_m_k.append(new_theta_m)
    theta_u_k.append(new_theta_u)
    beta0_k.append(beta0)
    beta1_k.append(beta1)
    alpha_k.append(new_alpha)
    sigma_square_k.append(new_sigma_square)
    mu2_k.append(new_mu2)
    sigma2_square_k.append(new_sigma2_square)

In [23]:
# QUESTION EST CE QU IL FAUT: NE PAS LES FAIRE TOUS EN MEME TEMPS MAIS 1 PAR 1


5.843309640884399

In [89]:
# INITIAL POSTERIOR

# likelihood 1
# need linked_records computed based on z
Y = np.array(linked_records_k[-1].Y)
X = np.array([linked_records_k[-1].propensity_score*linked_records_k[-1].treatment, linked_records_k[-1].propensity_score])
X = sm.add_constant(X)
X = X.T
model = sm.GLM(Y,X)
results = model.fit()
residuals = Y - X @ results.params
estimated_variance = residuals.T @ residuals / (len(residuals) - (X.shape[1]+1))
np.log(scipy.stats.norm.pdf(residuals, 0, np.sqrt(estimated_variance))).sum()

# likelihood 2
# need z
Y = scipy.stats.norm.rvs(mu2_k[-1], np.sqrt(sigma2_square_k[-1]), size=len(z_k[-1][z_k[-1]<0]))
np.log(scipy.stats.norm.pdf(Y, mu2_k[-1], np.sqrt(sigma2_square_k[-1]))).sum()

# likelihood 3 and 4
# need AB, z, theta_m, theta_u ATTENTION AB SHOULD BE KNOWN (GLOBAL)
idx_A = z_k[-1][z_k[-1]>=0]
idx_B = np.nonzero(z_k[-1]>=0)[0]
links = pds.MultiIndex.from_tuples(zip(idx_A,idx_B))
pairs = pds.MultiIndex.from_frame(AB[["source_index_A", "source_index_B"]])
# 3
data = AB[pairs.isin(links)]
pattern_match, count_match = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
((pattern_match @ np.log(theta_m_k[-1]) + (1-pattern_match) @ np.log(1-theta_m_k[-1])) * count_match).sum()
# 4
data = AB[~pairs.isin(links)]
pattern_unmatch, count_unmatch = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
((pattern_unmatch @ np.log(theta_u_k[-1]) + (1-pattern_unmatch) @ np.log(1-theta_u_k[-1])) * count_unmatch).sum()

# prior 1
# need z, alpha_pi, beta_pi
n_AB = (z_k[-1]>=0).sum()
math.log(math.factorial(A.shape[0]-n_AB)) - math.log(math.factorial(A.shape[0])) + scipy.special.betaln(n_AB + alpha_pi_k[-1], B.shape[0] - n_AB + beta_pi_k[-1]) - scipy.special.betaln(alpha_pi_k[-1], beta_pi_k[-1])

# prior 2
# need theta_m QUESTION a-1 / b-1?
(1 * np.log(theta_m_k[-1]) + 1 * np.log(1-theta_m_k[-1])).sum()

# prior 3
# need theta_u QUESTION a-1 / b-1?
(1 * np.log(theta_u_k[-1]) + 1 * np.log(1-theta_u_k[-1])).sum()

# prior 4
# need sigma_square, a_sigma, b_sigma QUESTION unclear if it is the pdf?
np.log(scipy.stats.invgauss.pdf(sigma_square_k[-1], a_sigma_k[-1], b_sigma_k[-1]))

# prior 5
# need beta0, beta1, alpha QUESTION unclear if it is the pdf?
np.log(scipy.stats.multivariate_normal.pdf([beta0_k[-1], beta1_k[-1], alpha_k[-1]], [0,0,0], np.eye(3)))

# prior 6
# need mu2 QUESTION unclear if it is the pdf?
np.log(scipy.stats.norm.pdf(mu2_k[-1], 0, 1))

# prior 7
# need sigma2_square, a_sigma2, b_sigma2 QUESTION unclear if it is the pdf?
np.log(scipy.stats.invgauss.pdf(sigma2_square_k[-1], a_sigma2_k[-1], b_sigma2_k[-1]))

-0.3593661589996669

In [90]:
# PROPOSALS

# z
linked_record_prop = (z_k[-1]<B.shape[0]).sum() / len(z_k[-1]) # proportion of linked records in previous z
new = np.random.choice(A.shape[0], size=B.shape[0], replace=False)
new = new * np.random.choice([1,-1], size=B.shape[0], p=[linked_record_prop, 1-linked_record_prop]) # randomly set negative some values so that the proportion of positive values corresponds to linked_record_prop
z_k.append(new)

# linked records
idx_A = z_k[-1][z_k[-1]>=0]
idx_B = np.nonzero(z_k[-1]>=0)[0]
from_A = A.iloc[idx_A,:].reset_index(drop=True)
from_B = B.iloc[idx_B,:].reset_index(drop=True)
linked_records = pds.concat([from_B, from_A.Y], axis=1)
linked_records['propensity_score'] = propensity_score(linked_records, covariates, None, False)
linked_records_k = [linked_records]

# theta_m QUESTION we do not care of dependence on previous param
links = pds.MultiIndex.from_tuples(zip(idx_A,idx_B))
pairs = pds.MultiIndex.from_frame(AB[["source_index_A", "source_index_B"]])
comparisons = AB[pairs.isin(links)].filter(regex="comparison")
match = comparisons.sum(axis=0) / len(comparisons)
theta_m_k.append(match)

# theta_u QUESTION we do not care of dependence on previous param
comparisons = AB.filter(regex="comparison")
unmatch = comparisons.sum(axis=0) / len(comparisons)
theta_u_k.append(unmatch)

# beta0, beta1, alpha, sigma_square
beta0_k.append(beta0_k[-1]+scipy.stats.norm.rvs(0,0.5))
beta1_k.append(beta1_k[-1]+scipy.stats.norm.rvs(0,0.5))
alpha_k.append(alpha_k[-1]+scipy.stats.norm.rvs(0,0.5))
sigma_square_k.append(sigma_square_k[-1]+scipy.stats.invgauss.rvs(a_sigma_k[-1],b_sigma_k[-1]))

# mu2, sigma2_square
mu2_k.append(mu2_k[-1]+scipy.stats.norm.rvs(0,1))
sigma2_square_k.append(sigma2_square_k[-1]+scipy.stats.invgauss.rvs(a_sigma2_k[-1],b_sigma2_k[-1]))

Optimization terminated successfully.
         Current function value: 0.218068
         Iterations 8


In [91]:
# NEW POSTERIOR:

# likelihood 1
# need linked_records computed based on z
Y = np.array(linked_records_k[-1].Y)
X = np.array([linked_records_k[-1].propensity_score*linked_records_k[-1].treatment, linked_records_k[-1].propensity_score])
X = sm.add_constant(X)
X = X.T
model = sm.GLM(Y,X)
results = model.fit()
residuals = Y - X @ results.params
estimated_variance = residuals.T @ residuals / (len(residuals) - (X.shape[1]+1))
np.log(scipy.stats.norm.pdf(residuals, 0, np.sqrt(estimated_variance))).sum()

# likelihood 2
# need z
Y = scipy.stats.norm.rvs(mu2_k[-1], np.sqrt(sigma2_square_k[-1]), size=len(z_k[-1][z_k[-1]<0]))
np.log(scipy.stats.norm.pdf(Y, mu2_k[-1], np.sqrt(sigma2_square_k[-1]))).sum()

# likelihood 3 and 4
# need AB, z, theta_m, theta_u ATTENTION AB SHOULD BE KNOWN (GLOBAL)
idx_A = z_k[-1][z_k[-1]>=0]
idx_B = np.nonzero(z_k[-1]>=0)[0]
links = pds.MultiIndex.from_tuples(zip(idx_A,idx_B))
pairs = pds.MultiIndex.from_frame(AB[["source_index_A", "source_index_B"]])
# 3
data = AB[pairs.isin(links)]
pattern_match, count_match = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
((pattern_match @ np.log(theta_m_k[-1]) + (1-pattern_match) @ np.log(1-theta_m_k[-1])) * count_match).sum()
# 4
data = AB[~pairs.isin(links)]
pattern_unmatch, count_unmatch = np.unique(data.filter(regex="comparison"), return_counts=True, axis=0)
((pattern_unmatch @ np.log(theta_u_k[-1]) + (1-pattern_unmatch) @ np.log(1-theta_u_k[-1])) * count_unmatch).sum()

# prior 1
# need z, alpha_pi, beta_pi
n_AB = (z_k[-1]>=0).sum()
math.log(math.factorial(A.shape[0]-n_AB)) - math.log(math.factorial(A.shape[0])) + scipy.special.betaln(n_AB + alpha_pi_k[-1], B.shape[0] - n_AB + beta_pi_k[-1]) - scipy.special.betaln(alpha_pi_k[-1], beta_pi_k[-1])

# prior 2
# need theta_m QUESTION a-1 / b-1?
(1 * np.log(theta_m_k[-1]) + 1 * np.log(1-theta_m_k[-1])).sum()

# prior 3
# need theta_u QUESTION a-1 / b-1?
(1 * np.log(theta_u_k[-1]) + 1 * np.log(1-theta_u_k[-1])).sum()

# prior 4
# need sigma_quare, a_sigma, b_sigma QUESTION unclear if it is the pdf?
np.log(scipy.stats.invgauss.pdf(sigma_square_k[-1], a_sigma_k[-1], b_sigma_k[-1]))

# prior 5
# need beta0, beta1, alpha QUESTION unclear if it is the pdf?
np.log(scipy.stats.multivariate_normal.pdf([beta0_k[-1], beta1_k[-1], alpha_k[-1]], [0,0,0], np.eye(3)))

# prior 6
# need mu2 QUESTION unclear if it is the pdf?
np.log(scipy.stats.norm.pdf(mu2_k[-1], 0, 1))

# prior 7
# need sigma2_square, a_sigma2, b_sigma2 QUESTION unclear if it is the pdf?
np.log(scipy.stats.invgauss.pdf(sigma2_square_k[-1], a_sigma2_k[-1], b_sigma2_k[-1]))

NameError: name 'theta_m' is not defined

In [ ]:
end = time.time()